In [1]:
import pandas as pd
import shutil, os
import numpy as np

In [2]:
df = pd.read_csv('fdr.csv')
df.head()

,parcel_name,sws_rs,sws_gb,sws_ms,sws_fa,sws_rw,sws_total
0,gmd_17Networks_17,NaN,NaN,NaN,NaN,0.048864,NaN
1,gmd_17Networks_1,NaN,NaN,NaN,NaN,0.042370,NaN
2,zReHo_17Networks_15,0.017524,NaN,NaN,NaN,0.041528,0.04454
3,zReHo_17Networks_11,0.016880,NaN,NaN,NaN,0.048864,NaN
4,zReHo_17Networks_9,0.024346,NaN,NaN,NaN,0.041528,NaN


In [3]:
df1 = df[['parcel_name' ,'sws_rs']]
df1.head()

,parcel_name,sws_rs
0,gmd_17Networks_17,NaN
1,gmd_17Networks_1,NaN
2,zReHo_17Networks_15,0.017524
3,zReHo_17Networks_11,0.016880
4,zReHo_17Networks_9,0.024346


In [4]:
df1.shape

(687, 2)

In [5]:
df2 = df1.dropna()

In [6]:
df2.shape

(325, 2)

In [7]:
df2.insert(1, 'parc_name', np.nan)

In [8]:
df2.head()

,parcel_name,parc_name,sws_rs
2,zReHo_17Networks_15,NaN,0.017524
3,zReHo_17Networks_11,NaN,0.016880
4,zReHo_17Networks_9,NaN,0.024346
5,zReHo_17Networks_7,NaN,0.039745
6,zReHo_17Networks_5,NaN,0.043626


In [9]:
df2 = df2.reset_index()

In [10]:
df2.head()

,index,parcel_name,parc_name,sws_rs
0,2,zReHo_17Networks_15,NaN,0.017524
1,3,zReHo_17Networks_11,NaN,0.016880
2,4,zReHo_17Networks_9,NaN,0.024346
3,5,zReHo_17Networks_7,NaN,0.039745
4,6,zReHo_17Networks_5,NaN,0.043626


In [11]:
df = df2.drop('index', axis = 1)

In [12]:
df.head()

,parcel_name,parc_name,sws_rs
0,zReHo_17Networks_15,NaN,0.017524
1,zReHo_17Networks_11,NaN,0.016880
2,zReHo_17Networks_9,NaN,0.024346
3,zReHo_17Networks_7,NaN,0.039745
4,zReHo_17Networks_5,NaN,0.043626


In [13]:
markers = ['gmd_', 'zALFF_', 'zfALFF_', 'zReHo_', 'zDCwei_', 'zDCbin_']
for i,parc in enumerate(df["parcel_name"]):
    #for i in range(df.shape[0]):
    for marker in markers:
        if parc.find(marker)==0:
            df["parc_name"][i] = df["parcel_name"][i][len(marker):]
            #print(df["sws_total"][i])
            col = marker + "sws_rs_fdr" 
            if col not in df:
                df.insert(2, col, np.nan)
            df.iloc[i, df.columns.get_loc(col)] = df["sws_rs"][i]

<ipython-input-13-4a178154a71b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["parc_name"][i] = df["parcel_name"][i][len(marker):]
/home/tara/Programming/env/lib/python3.8/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
#df.to_csv('fdr_reformed.csv')
df.shape

(325, 9)

### merging duplicates

In [19]:
#df = pd.read_csv('fdr_reformed.csv')
#df.head()

In [ ]:
#df.shape

In [17]:
df2 = df.copy()
inds = df2['parc_name'].drop_duplicates().reset_index(drop=True)
df2.set_index('parc_name', inplace=True)
df3 = pd.DataFrame(columns = df2.columns)
for ind in inds:
    region = df2.loc[[ind]]
    if region.shape[0] > 1 :
        reg = region
        cols = reg.columns
        for col in cols:
            reg[col] = reg[col].sum()
        reg = reg.drop_duplicates()
        df3 = pd.concat([df3, reg])
        continue
    df3 = pd.concat([df3, region])

In [18]:
df3.shape

(216, 8)

In [19]:
df3.head()

,parcel_name,gmd_sws_rs_fdr,zALFF_sws_rs_fdr,zDCbin_sws_rs_fdr,zDCwei_sws_rs_fdr,zfALFF_sws_rs_fdr,zReHo_sws_rs_fdr,sws_rs
17Networks_15,zReHo_17Networks_15,NaN,NaN,NaN,NaN,NaN,0.017524,0.017524
17Networks_11,zReHo_17Networks_11zfALFF_17Networks_11zDCwei_...,0.0,0.0,0.028364,0.024346,0.029930,0.016880,0.099519
17Networks_9,zReHo_17Networks_9zfALFF_17Networks_9zDCwei_17...,0.0,0.0,0.034696,0.026202,0.024832,0.024346,0.110076
17Networks_7,zReHo_17Networks_7,NaN,NaN,NaN,NaN,NaN,0.039745,0.039745
17Networks_5,zReHo_17Networks_5zfALFF_17Networks_5zDCwei_17...,0.0,0.0,0.000000,0.049275,0.039609,0.043626,0.132509


In [23]:
#df3 = df3.reset_index()

In [24]:
#df3.head()

,index,parcel_name,gmd_sws_total_fdr,zALFF_sws_total_fdr,zDCbin_sws_total_fdr,zDCwei_sws_total_fdr,zfALFF_sws_total_fdr,zReHo_sws_total_fdr,sws_total
0,17Networks_15,zReHo_17Networks_15,NaN,NaN,NaN,NaN,NaN,0.044540,0.044540
1,17Networks_9,zfALFF_17Networks_9zDCwei_17Networks_9zDCbin_1...,0.0,0.000000,0.048667,0.04454,0.044478,0.000000,0.137685
2,17Networks_2,zfALFF_17Networks_2zALFF_17Networks_2,0.0,0.041309,0.000000,0.00000,0.049611,0.000000,0.090920
3,17Networks_1,zALFF_17Networks_1,NaN,0.009779,NaN,NaN,NaN,NaN,0.009779
4,cHipp_L,zReHo_cHipp_LzfALFF_cHipp_L,0.0,0.000000,0.000000,0.00000,0.014855,0.039714,0.054570


In [20]:
df3.to_csv('SWSrs_fdr_reformed.csv')

In [ ]:
df3.shape